# Merge and keep unique data from admissions and patient tables  

In [1]:
import gc
import gzip
import pandas as pd

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Diagnosis file 
diagnosis_path = 'CSV\Exports\o01_eicu_diagnosis.csv'

compressed_file_path = r"..\Datasets\eicu-2_0\patient.csv.gz"

df = pd.read_csv(compressed_file_path, compression='gzip')

# Read the specific ICD codes from a CSV file
diagnosis_df = pd.read_csv(diagnosis_path)

In [2]:
# Columns related to ICU
icu_columns = [
    'uniquepid', # ID for a unique patient
    'unitvisitnumber', # identifies the visit number of the patient
    'patienthealthsystemstayid', # surrogate key for hospital Stay
    'patientunitstayid', # surrogate key for ICU Stay
    'gender', # gender of the patient
    'age', # age of the patient in full years
    'ethnicity', # ethnicity of the patient
    'wardid', # surrogate key for the ward associated with the patient unit stay
    'apacheadmissiondx', #Full path string of admission diagnosis for patients unit stay
    'admissionheight', # admission height of the patient in cm
    'unittype', # the picklist unit type of the unit
    'unitadmittime24', # time in 24 hour format of when the unit admit event occurred
    'unitadmitsource', # picklist location from where the patient was admitted
    'admissionweight', #admission weight of the patient in kilograms
    'dischargeweight', # patient weight at time of unit discharge in kilograms 
    'unitdischargetime24', # time in 24 hour format of when the unit discharge event occurred
    'unitdischargeoffset', # number of minutes from unit admit time that the patient was discharged from the unit
    'unitdischargelocation', #  Structured list of locations where the patient was discharged to from the unit
    'unitdischargestatus' # specifies patient’s condition upon leaving the unit
]

# Create a new DataFrame with only the ICU-related columns
temp_df = df[icu_columns]

In [3]:
# Merge dataframes
merged_df = pd.merge(diagnosis_df, df, on='patientunitstayid', how='inner')

# Move 'uniquepid' column to the beginning
uniquepid_column = merged_df.pop('uniquepid')
merged_df.insert(0, 'uniquepid', uniquepid_column)


# Sort the DataFrame by uniquepid and hospitalAdmitOffset
sorted_table = merged_df.sort_values(by=['uniquepid', 'hospitaladmitoffset'])

In [4]:
# Create an additional column to represent the order of appearance
sorted_table['order_of_appearance'] = sorted_table.groupby('uniquepid').cumcount()

In [5]:
# Group by 'uniquepid' and get the index of the row with the maximum 'order_of_appearance'
idx = sorted_table.groupby('uniquepid')['order_of_appearance'].idxmax()

# Select the corresponding rows and reset index
result = sorted_table.loc[idx].reset_index(drop=True)

In [6]:
# Export the filtered DataFrame to a CSV file
result.to_csv('CSV\Exports\o02_eicu_unique_admissions.csv', index=False, header=True)

# Free RAM
gc.collect()

0

In [7]:
display (result)

,uniquepid,diagnosisid,patientunitstayid,activeupondischarge,diagnosisoffset,diagnosisstring,icd9code,diagnosispriority,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeyear,hospitaldischargetime24,hospitaldischargeoffset,hospitaldischargelocation,hospitaldischargestatus,unittype,unitadmittime24,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,order_of_appearance
0,002-11314,3623319,204938,True,20,neurologic|disorders of vasculature|stroke|isc...,434.91,Primary,178465,Male,83,African American,73,99,"Renal failure, acute",180.3,00:13:00,-191,Emergency Department,2014,16:40:00,10876,Home,Alive,Med-Surg ICU,03:24:00,Emergency Department,1,admit,NaN,57.40,20:35:00,2471,Floor,Alive,0
1,002-12035,4113259,198483,True,125,neurologic|disorders of vasculature|stroke|isc...,434.91,Primary,173433,Female,61,Caucasian,73,102,"CVA, cerebrovascular accident/stroke",162.6,02:38:00,-2,Emergency Department,2014,17:13:00,8073,Home,Alive,Neuro ICU,02:40:00,Emergency Department,1,admit,71.4,75.00,20:57:00,3977,Floor,Alive,0
2,002-12588,3500893,213211,True,13,neurologic|disorders of vasculature|TIA,435.9,Primary,184879,Male,44,Caucasian,63,95,"CVA, cerebrovascular accident/stroke",172.7,10:13:00,-4,Emergency Department,2014,22:04:00,5027,Home,Alive,Med-Surg ICU,10:17:00,Emergency Department,1,admit,108.3,109.70,00:49:00,872,Floor,Alive,0
3,002-13204,3975123,169514,False,2303,neurologic|disorders of vasculature|TIA,435.9,Other,150818,Female,36,Caucasian,73,102,"CVA, cerebrovascular accident/stroke",170.2,15:14:00,-40,NaN,2014,19:23:00,10289,Home,Alive,Neuro ICU,15:54:00,Emergency Department,1,admit,74.3,73.70,22:16:00,4702,Floor,Alive,0
4,002-13426,3752191,216904,False,19497,neurologic|disorders of vasculature|stroke|hem...,430,Other,187745,Male,74,Caucasian,73,102,Head only trauma,177.8,20:57:00,-2,Other Hospital,2014,18:15:00,47356,Skilled Nursing Facility,Alive,Neuro ICU,20:59:00,Operating Room,1,admit,74.8,88.10,14:16:00,31277,Other ICU,Alive,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,035-9170,46348931,3342242,False,29,neurologic|disorders of vasculature|stroke|isc...,434.91,Primary,2733170,Male,71,Caucasian,458,1104,"Hemorrhage/hematoma, intracranial",185.4,17:37:00,-127,Emergency Department,2014,21:39:00,10195,Death,Expired,Cardiac ICU,19:44:00,Emergency Department,1,admit,124.4,131.80,21:39:00,10195,Death,Expired,0
5417,035-9406,46348775,3328265,True,10957,neurologic|disorders of vasculature|stroke|isc...,434.91,Other,2720588,Female,70,Caucasian,458,1109,Seizures (primary-no structural brain disease),162.6,19:58:00,-83,Emergency Department,2014,19:01:00,15700,Home,Alive,Cardiac ICU,21:21:00,Emergency Department,1,admit,63.1,58.80,02:31:00,11830,Telemetry,Alive,0
5418,035-9442,46347272,3350395,True,88,neurologic|disorders of vasculature|stroke|isc...,434.91,Other,2740541,Female,69,Caucasian,459,1108,"CVA, cerebrovascular accident/stroke",167.6,17:46:00,-219,Emergency Department,2015,22:01:00,2916,Home,Alive,Med-Surg ICU,21:25:00,Emergency Department,1,admit,95.5,95.50,19:36:00,1331,Telemetry,Alive,0
5419,035-9445,46351788,3340374,True,314,neurologic|disorders of vasculature|TIA,435.9,Other,2731472,Female,65,African American,458,1109,Coma/change in level of consciousness (for hep...,157.5,16:23:00,-2899,Step-Down Unit (SDU),2015,21:54:00,17592,Rehabilitation,Alive,Cardiac ICU,16:42:00,Step-Down Unit (SDU),1,admit,78.1,75.70,22:29:00,3227,Telemetry,Alive,0
